<a href="https://colab.research.google.com/github/GiovanniSorice/Hate_Speech_Detection/blob/main/AlBerto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AlBERTo Hate Speech Classifier 

In [2]:
!pip install ekphrasis
!pip install bert-tensorflow
!pip install transformers==3.5


import pandas as pd
import numpy as np
import collections
import logging
import os
import re
import logger

from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons
from transformers import BertTokenizer, WordpieceTokenizer
from transformers.tokenization_bert import load_vocab

In [3]:
from google.colab import drive
# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Load the dataset 

In [34]:
# directory name 
input_dir = '/content/drive/My Drive/HLT/clean_dataset_training/' 
AlBERTo_path = '/content/drive/MyDrive/HLT/alberto_uncased_L-12_H-768_A-12_italian_ckpt/'

# Spec
pd.set_option("display.max_colwidth", None)

In [35]:
tsv_file = open(input_dir+"training_dataset.csv")

dataset = pd.read_csv(tsv_file,sep=',')

## Configure AlBERTo classes 

In [8]:
class AlBERTo_Preprocessing(object):
    def __init__(self, do_lower_case=True, **kwargs):
        self.do_lower_case = do_lower_case

    def preprocess(self, text):
        if self.do_lower_case:
            text = text.lower()
        text = str(" ".join(text_processor.pre_process_doc(text)))
        text = re.sub(r'[^a-zA-ZÀ-ú</>!?♥♡\s\U00010000-\U0010ffff]', ' ', text)
        text = re.sub(r'\s+', ' ', text)
        text = re.sub(r'(\w)\1{2,}', r'\1\1', text)
        text = re.sub(r'^\s', '', text)
        text = re.sub(r'\s$', '', text)
        return text

In [9]:
class CharacterTokenizer(object):
    """Runs Character tokenziation."""

    def __init__(self, vocab, unk_token,
                 max_input_chars_per_word=100, with_markers=True):
        """Constructs a CharacterTokenizer.
        Args:
            vocab: Vocabulary object.
            unk_token: A special symbol for out-of-vocabulary token.
            with_markers: If True, "#" is appended to each output character except the
                first one.
        """
        self.vocab = vocab
        self.unk_token = unk_token
        self.max_input_chars_per_word = max_input_chars_per_word
        self.with_markers = with_markers

    def tokenize(self, text):
        """Tokenizes a piece of text into characters.
        For example:
            input = "apple"
            output = ["a", "##p", "##p", "##l", "##e"]  (if self.with_markers is True)
            output = ["a", "p", "p", "l", "e"]          (if self.with_markers is False)
        Args:
            text: A single token or whitespace separated tokens.
                This should have already been passed through `BasicTokenizer`.
        Returns:
            A list of characters.
        """

        output_tokens = []
        for i, char in enumerate(text):
            if char not in self.vocab:
                output_tokens.append(self.unk_token)
                continue

            if self.with_markers and i != 0:
                output_tokens.append('##' + char)
            else:
                output_tokens.append(char)

        return output_tokens

## Tokenize dataset 

In [24]:
vocab_file_path = AlBERTo_path+"vocab.txt"

tokenizer = AlBERToTokenizer(do_lower_case=True,vocab_file=vocab_file_path)

AttributeError: ignored

In [21]:
dataset['text'] = dataset['text'].apply(tokenizer.tokenize)

AttributeError: ignored

In [28]:
from tokenizer import *
from transformers import AutoTokenizer, AutoModel

model = AutoModel.from_pretrained("m-polignano-uniba/bert_uncased_L-12_H-768_A-12_italian_alb3rt0")

[26/Feb/2021 15:48:18] INFO - Lock 140075500667280 acquired on /root/.cache/torch/transformers/ddbe3a15a091d937ecbe95bd9c6fc3c82a8c0fde2b2725504806bae3d5b553c7.ae4dca69541bd7de0c171dd17916268b75b9de1cdbd9670add7a62ad5b98a01e.lock



[26/Feb/2021 15:48:19] INFO - Lock 140075500667280 released on /root/.cache/torch/transformers/ddbe3a15a091d937ecbe95bd9c6fc3c82a8c0fde2b2725504806bae3d5b553c7.ae4dca69541bd7de0c171dd17916268b75b9de1cdbd9670add7a62ad5b98a01e.lock
[26/Feb/2021 15:48:19] INFO - Lock 140075501099728 acquired on /root/.cache/torch/transformers/8270b211710cba9f85a4fd7242e31b37d94e76042d67de09b6eac7ab6e7ef78e.48dbff3c9ce886c188677656cbfb17029a8a066539b2da658b60d42766b0122c.lock



[26/Feb/2021 15:48:39] INFO - Lock 140075501099728 released on /root/.cache/torch/transformers/8270b211710cba9f85a4fd7242e31b37d94e76042d67de09b6eac7ab6e7ef78e.48dbff3c9ce886c188677656cbfb17029a8a066539b2da658b60d42766b0122c.lock


In [33]:


tok = AutoTokenizer.from_pretrained("m-polignano-uniba/bert_uncased_L-12_H-768_A-12_italian_alb3rt0")
tokens = tok.tokenize("notizia del giorno sono morto")
print(tokens)

['notizia', 'del', 'giorno', 'sono', 'morto']


In [31]:
dataset['text'] = dataset['text'].apply(tokenizer.tokenize)

In [36]:
dataset

,id,text,hs,stereotype,text_length,#C-L words,hashtags,#?!,"#.,",tokens,#bad_words,%bad_words
0,2066,È terrorismo anche questo per mettere in uno stato di soggezione le persone e renderle innocue mentre qualcuno,0,0,120,10,NaN,0,5,"['È', 'terrorismo', 'anche', 'questo', 'per', 'mettere', 'in', 'uno', 'stato', 'di', 'soggezione', 'le', 'persone', 'e', 'renderle', 'innocue', 'mentre', 'qualcuno']",0,0
1,2045,infatti finché ci hanno guadagnato con i campi rom tutto era ok con alemanno ipocriti,0,0,101,0,"['#rom', '#Alemanno', '#Ipocriti']",0,0,"['infatti', 'finché', 'ci', 'hanno', 'guadagnato', 'con', 'i', 'campi', 'rom', 'tutto', 'era', 'ok', 'con', 'alemanno', 'ipocriti']",1,6
2,61,Corriere Tangenti Mafia Capitale dimenticataMazzette su buche e campi rom rom a,0,0,86,8,['#roma'],0,1,"['Corriere', 'Tangenti', 'Mafia', 'Capitale', 'dimenticataMazzette', 'su', 'buche', 'e', 'campi', 'rom', 'rom', 'a']",3,25
3,1259,ad uno ad uno perché quando i migranti israeliti arrivarono in terra di Canaan fecero fuori tutti i Cananiti,0,0,118,0,NaN,0,2,"['ad', 'uno', 'ad', 'uno', 'perché', 'quando', 'i', 'migranti', 'israeliti', 'arrivarono', 'in', 'terra', 'di', 'Canaan', 'fecero', 'fuori', 'tutti', 'i', 'Cananiti']",0,0
4,949,Il divertimento del giorno Trovare i patrioti italiani che inneggiano contro i rom facendo la spesa alla li dl multinazionale tedesca,0,0,138,0,['#Lidl'],1,1,"['Il', 'divertimento', 'del', 'giorno', 'Trovare', 'i', 'patrioti', 'italiani', 'che', 'inneggiano', 'contro', 'i', 'rom', 'facendo', 'la', 'spesa', 'alla', 'li', 'dl', 'multinazionale', 'tedesca']",1,4
...,...,...,...,...,...,...,...,...,...,...,...,...
6832,9340,Gli stati nazionali devono essere pronti a rinunciare alla propria sovranità Lo ha detto la Merkel che ha aggiunto che gli stati nazionali non devono ascoltare la volontà dei loro cittadini quando si tratta di questioni che riguardano immigrazione confini o persino sovranità,0,0,285,2,NaN,0,4,"['Gli', 'stati', 'nazionali', 'devono', 'essere', 'pronti', 'a', 'rinunciare', 'alla', 'propria', 'sovranità', 'Lo', 'ha', 'detto', 'la', 'Merkel', 'che', 'ha', 'aggiunto', 'che', 'gli', 'stati', 'nazionali', 'non', 'devono', 'ascoltare', 'la', 'volontà', 'dei', 'loro', 'cittadini', 'quando', 'si', 'tratta', 'di', 'questioni', 'che', 'riguardano', 'immigrazione', 'confini', 'o', 'persino', 'sovranità']",0,0
6833,9121,Il ministro dellinterno della Germania HorstSehofersta facendo la proposta di dare soldi agli immigrati che vogliono tornare a casa e aiutarli a creare unattività a casa loro e fare busines con la GermaniaChi pagaUna parte i crucchi e il resto lEuropa cioè io e voi,0,0,277,0,"['#HorstSeehofer,sta']",2,3,"['Il', 'ministro', 'dellinterno', 'della', 'Germania', 'HorstSehofersta', 'facendo', 'la', 'proposta', 'di', 'dare', 'soldi', 'agli', 'immigrati', 'che', 'vogliono', 'tornare', 'a', 'casa', 'e', 'aiutarli', 'a', 'creare', 'unattività', 'a', 'casa', 'loro', 'e', 'fare', 'busines', 'con', 'la', 'GermaniaChi', 'pagaUna', 'parte', 'i', 'crucchi', 'e', 'il', 'resto', 'lEuropa', 'cioè', 'io', 'e', 'voi']",0,0
6834,8549,Salvini In Italia troppi si sono montati la testa io ringrazio Dio e voi per questi mesi straordinari Vi raccontavano che su immigrazione non si poteva fare nulla è bastato usare buonsenso e coraggio io ci sono piazza del popolo,0,0,233,0,"['#Salvini:', '#iocisono', '#piazzadelpopolo']",0,4,"['Salvini', 'In', 'Italia', 'troppi', 'si', 'sono', 'montati', 'la', 'testa', 'io', 'ringrazio', 'Dio', 'e', 'voi', 'per', 'questi', 'mesi', 'straordinari', 'Vi', 'raccontavano', 'che', 'su', 'immigrazione', 'non', 'si', 'poteva', 'fare', 'nulla', 'è', 'bastato', 'usare', 'buonsenso', 'e', 'coraggio', 'io', 'ci', 'sono', 'piazza', 'del', 'popolo']",0,0
6835,9240,Chi giubila in buona fede non ha capito niente Purtroppo credo che i più non siano in buona fede I migranti sono un grosso busines e chi finora li ha voluti non vuole perdere questo guadagno,0,0,206,2,NaN,0,2,"['Chi', 'giubila', 'in', 'buona', 'fede